# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [14]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [15]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [17]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:
        # ...
        #se è true devo eseguire la PCA
        # standardizzazione
        scaler=StandardScaler()
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_val=scaler.transform(X_val)
        
        #PCA
        n_dimensioni=hyperparams['pca_components']
        pca=PCA(n_dimensioni)
        X_train=pca.fit_transform(X_train)
        X_val=pca.transform(X_val)
        
    if hyperparams['data_standardize']:
        # ...
        #standardizzo di nuovo nonostante lo possa aver già fatto sopra?
        scaler=StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_val = scaler.transform(X_val)
        
    # Aggiunge il termine costante ai dati
    # ...
    #creo i vettori colonna di 1
    v1_train=np.ones(X_train.shape[0])
    v1_val=np.ones(X_val.shape[0])
    #poi concateno
    X_train=np.c_[v1_train,X_train]
    X_val=np.c_[v1_val,X_val]

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        # ...
        l=hyperparams['reg_lambda'] 
        m_identita=np.identity(X_train.shape[1])
        parametri=np.linalg.inv(X_train.T@X_train+l*m_identita)@X_train.T@y_train
        
    else:
        # ...
        #altrimenti uso la stessa formula senza la regolarizzazione
        parametri=np.linalg.inv(X_train.T@X_train)@X_train.T@y_train


    # Calcolo predizioni
    # ...
    y_predizioni_train=X_train@parametri
    y_predizioni_val=X_val@parametri

    # Calcola il MAE
    # ...
    mae_train = np.mean(np.abs(y_train - y_predizioni_train))
    mae_val = np.mean(np.abs(y_val - y_predizioni_val))
    return mae_train,mae_val,parametri

# Dividi il dataset in training e test set
# Dividi il training set in training set effettivo e validation set

train_fraction=0.6
val_fraction=0.2
test_fraction=0.2

n_tot=X.shape[0]
n_train=int(n_tot*train_fraction)
n_val=int(n_tot*val_fraction)
n_test=int(n_tot*test_fraction)

X_train=X[:n_train]
y_train=y[:n_train]
X_val=X[n_train:n_train+n_val]
y_val=y[n_train:n_train+n_val]
X_test=X[n_train+n_val:]
y_test=y[n_train+n_val:]

    
# Trova la configurazione di iperparametri migliore
# ... faccio un ciclo for per testare tutte le configurazioni e ad ogni loop mi calcolo
# le mae di train e val, segnandomi quali sono le migliori 

best_mae_val=float('inf')
best_mae_train=float('inf')
best_config=None
best_parametri=None


for config in configs:
    mae_train, mae_val, parametri = pipeline(X_train.copy(), y_train.copy(), X_val.copy(), y_val.copy(), config)
    if mae_val < best_mae_val: 
        best_mae_val = mae_val
        best_mae_train = mae_train
        best_config = config
        best_parametri = parametri
        
# Riallena il modello sul training set completo
# ...

X_train_completo = np.concatenate([X_train, X_val], axis=0)
y_train_completo = np.concatenate([y_train, y_val], axis=0)

mae_train_finale, mae_val_finale, parametri_definitivi = pipeline(X_train_completo.copy(), y_train_completo.copy(),X_test.copy(), y_test.copy(),best_config)


# Calcola il MAE sul test set
# ...
v1_test = np.ones(X_test.shape[0])
X_test = np.c_[v1_test, X_test]
y_predizioni_test = X_test @ parametri_definitivi
mae_test = np.mean(np.abs(y_test - y_predizioni_test))



# Stampa  risultati
# ...
print("Migliore configurazione:", best_config)
print("mae di train:", best_mae_train)
print("mae di validation:", best_mae_val)
print("mae di train finale:", mae_train_finale)
print("mae di test:", mae_test)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Migliore configurazione: {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
mae di train: 0.5975177708822282
mae di validation: 0.5872605946947299
mae di train finale: 0.5942737135129479
mae di test: 7.050595902994301


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
